In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import urlopen
from bs4 import BeautifulSoup
%matplotlib inline

## Module 1: Scrapping the WEB

In [20]:
url = "http://www.senat.fr/communiques/"
html = urlopen(url)

In [21]:
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [22]:
# Get the title
title = soup.title
print(title)

<title>Sénat - Communiqués de presse</title>


In [1]:
# Print out the text
text = soup.get_text()
print(soup.text)

In [24]:
anchors = soup.find_all('a')
all_links = []
press_anchor_date_links = []
press_links = []
for links in anchors:
    if "presse" in str(links):        
        all_links.append(links.get("href"))
        press_anchor_date_links.append(str(links.get_text()))
        press_links = []
        for link in all_links:
            press_links.append("http://www.senat.fr"+str(link))
all_links = np.array(all_links)
#all_links

In [25]:
df_date = pd.DataFrame(press_anchor_date_links,columns=["Press Date"])
df_date

,Press Date
0,Espace presse
1,2019
2,2018
3,2017
4,2016
...,...
301,08 janvier 2020
302,08 janvier 2020
303,08 janvier 2020
304,08 janvier 2020


In [26]:
df_link = pd.DataFrame(press_links,columns=["Press Links"])
df_link["Press Links"]

0      http://www.senat.fr/espace_presse/index.html
1       http://www.senat.fr/presse/communi2019.html
2       http://www.senat.fr/presse/communi2018.html
3       http://www.senat.fr/presse/communi2017.html
4       http://www.senat.fr/presse/communi2016.html
                           ...                     
301     http://www.senat.fr/presse/cp20200108c.html
302     http://www.senat.fr/presse/cp20200108b.html
303     http://www.senat.fr/presse/cp20200108a.html
304      http://www.senat.fr/presse/cp20200108.html
305                            http://www.senat.fr#
Name: Press Links, Length: 306, dtype: object

In [27]:
press_data = pd.concat([df_link, df_date], axis = 1)
press_data

,Press Links,Press Date
0,http://www.senat.fr/espace_presse/index.html,Espace presse
1,http://www.senat.fr/presse/communi2019.html,2019
2,http://www.senat.fr/presse/communi2018.html,2018
3,http://www.senat.fr/presse/communi2017.html,2017
4,http://www.senat.fr/presse/communi2016.html,2016
...,...,...
301,http://www.senat.fr/presse/cp20200108c.html,08 janvier 2020
302,http://www.senat.fr/presse/cp20200108b.html,08 janvier 2020
303,http://www.senat.fr/presse/cp20200108a.html,08 janvier 2020
304,http://www.senat.fr/presse/cp20200108.html,08 janvier 2020


## Module 2: Importing data into a NoSQL database (MongoDB)

In [48]:
from pymongo import MongoClient
from random import randint
#Step 1: Connect to MongoDB - Note: Change connection string as needed to reflect your own connection string
client = MongoClient("mongodb+srv://<<username>>:<<password>>@cluster0-6jkfh.mongodb.net/<<DataBase>>?retryWrites=true&w=majority")
db=client.pressData
#Step 2: Create sample data
df = pd.DataFrame({"Press Date":np.array(press_data['Press Date']),"Press Link":np.array(press_data['Press Links'])},index=press_data['Press Date'])

# press_data.set_index('Press Date',inplace=True)
data_dict = df.to_dict("records")
data_dict

#Step 3: Insert business object directly into MongoDB via isnert_one
result=db.reviews.insert_many(data_dict)
print("Done!")

Done!
